# CO2 Emission Prediction on different Car Models

In [93]:
import altair as alt
import numpy as np
import pandas as pd
import string
from sklearn import tree
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_transformer 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import (
FunctionTransformer,
Normalizer,
OneHotEncoder,
StandardScaler,
normalize,
scale)
from sklearn.metrics import classification_report
from sklearn.svm import SVC, SVR
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, make_scorer
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score

from scipy.stats import lognorm, loguniform, randint
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, Ridge
import sklearn.metrics
import scipy
from scipy.stats import reciprocal, uniform
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

In [38]:
car = pd.read_csv('FuelConsumption.csv')
car.head()

,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,2014,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,2014,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,2014,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,2014,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,2014,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


In [39]:
car.info()
print('\n')
car.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   MODELYEAR                 1067 non-null   int64  
 1   MAKE                      1067 non-null   object 
 2   MODEL                     1067 non-null   object 
 3   VEHICLECLASS              1067 non-null   object 
 4   ENGINESIZE                1067 non-null   float64
 5   CYLINDERS                 1067 non-null   int64  
 6   TRANSMISSION              1067 non-null   object 
 7   FUELTYPE                  1067 non-null   object 
 8   FUELCONSUMPTION_CITY      1067 non-null   float64
 9   FUELCONSUMPTION_HWY       1067 non-null   float64
 10  FUELCONSUMPTION_COMB      1067 non-null   float64
 11  FUELCONSUMPTION_COMB_MPG  1067 non-null   int64  
 12  CO2EMISSIONS              1067 non-null   int64  
dtypes: float64(4), int64(4), object(5)
memory usage: 108.5+ KB




,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
count,1067.0,1067,1067,1067,1067.000000,1067.000000,1067,1067,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000
unique,NaN,39,663,16,NaN,NaN,22,4,NaN,NaN,NaN,NaN,NaN
top,NaN,FORD,F150 FFV,MID-SIZE,NaN,NaN,A6,X,NaN,NaN,NaN,NaN,NaN
freq,NaN,90,8,178,NaN,NaN,222,514,NaN,NaN,NaN,NaN,NaN
mean,2014.0,NaN,NaN,NaN,3.346298,5.794752,NaN,NaN,13.296532,9.474602,11.580881,26.441425,256.228679
std,0.0,NaN,NaN,NaN,1.415895,1.797447,NaN,NaN,4.101253,2.794510,3.485595,7.468702,63.372304
min,2014.0,NaN,NaN,NaN,1.000000,3.000000,NaN,NaN,4.600000,4.900000,4.700000,11.000000,108.000000
25%,2014.0,NaN,NaN,NaN,2.000000,4.000000,NaN,NaN,10.250000,7.500000,9.000000,21.000000,207.000000
50%,2014.0,NaN,NaN,NaN,3.400000,6.000000,NaN,NaN,12.600000,8.800000,10.900000,26.000000,251.000000
75%,2014.0,NaN,NaN,NaN,4.300000,8.000000,NaN,NaN,15.550000,10.850000,13.350000,31.000000,294.000000


In [40]:
car.drop(columns=['MODELYEAR'], inplace=True)

# Rename the columns for readability
car.columns = ['Make', 'Model', 'Vehicle_Class', 'Engine_Size', 'Cylinders', 'Transmission', 'Fuel_Type', 
               'Fuel_Consumption_City', 'Fuel_Consumption_Hwy', 'Fuel_Consumption_Comb', 'Fuel_Consumption_Comb_MPG', 'CO2_Emissions']
car

,Make,Model,Vehicle_Class,Engine_Size,Cylinders,Transmission,Fuel_Type,Fuel_Consumption_City,Fuel_Consumption_Hwy,Fuel_Consumption_Comb,Fuel_Consumption_Comb_MPG,CO2_Emissions
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244
...,...,...,...,...,...,...,...,...,...,...,...,...
1062,VOLVO,XC60 AWD,SUV - SMALL,3.0,6,AS6,X,13.4,9.8,11.8,24,271
1063,VOLVO,XC60 AWD,SUV - SMALL,3.2,6,AS6,X,13.2,9.5,11.5,25,264
1064,VOLVO,XC70 AWD,SUV - SMALL,3.0,6,AS6,X,13.4,9.8,11.8,24,271
1065,VOLVO,XC70 AWD,SUV - SMALL,3.2,6,AS6,X,12.9,9.3,11.3,25,260


#### Feature Engineering:

In [41]:
# Calculate fuel consumption per cylinder
car['Fuel_Consumption_Per_Cylinder'] = car['Fuel_Consumption_Comb'] / car['Cylinders']

# Calculate CO2 emissions per cylinder
car['CO2_Emissions_Per_Cylinder'] = car['CO2_Emissions'] / car['Cylinders']

# Calculate the ratio of engine size to cylinders
car['Engine_Size_Per_Cylinder'] = car['Engine_Size'] / car['Cylinders']

car

,Make,Model,Vehicle_Class,Engine_Size,Cylinders,Transmission,Fuel_Type,Fuel_Consumption_City,Fuel_Consumption_Hwy,Fuel_Consumption_Comb,Fuel_Consumption_Comb_MPG,CO2_Emissions,Fuel_Consumption_Per_Cylinder,CO2_Emissions_Per_Cylinder,Engine_Size_Per_Cylinder
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196,2.125000,49.000000,0.500000
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221,2.400000,55.250000,0.600000
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136,1.475000,34.000000,0.375000
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255,1.850000,42.500000,0.583333
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244,1.766667,40.666667,0.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,VOLVO,XC60 AWD,SUV - SMALL,3.0,6,AS6,X,13.4,9.8,11.8,24,271,1.966667,45.166667,0.500000
1063,VOLVO,XC60 AWD,SUV - SMALL,3.2,6,AS6,X,13.2,9.5,11.5,25,264,1.916667,44.000000,0.533333
1064,VOLVO,XC70 AWD,SUV - SMALL,3.0,6,AS6,X,13.4,9.8,11.8,24,271,1.966667,45.166667,0.500000
1065,VOLVO,XC70 AWD,SUV - SMALL,3.2,6,AS6,X,12.9,9.3,11.3,25,260,1.883333,43.333333,0.533333


#### Exploratory Data Analysis for correlation between various features with the target feature (CO2_Emissions)

In [42]:
scatter_plot = alt.Chart(car).mark_circle().encode(
    x='Engine_Size',
    y='CO2_Emissions',
    color='Cylinders:N',
    tooltip=['Engine_Size', 'Cylinders', 'CO2_Emissions']
).properties(
    width=600,
    height=400
).interactive()

# Show the scatter plot
scatter_plot

alt.Chart(...)

In [43]:
CO2_plot = alt.Chart(car).mark_circle().encode(
    x=alt.X('Fuel_Consumption_City', title='Fuel Consumption City (L/100km)'),
    y=alt.Y('Fuel_Consumption_Hwy', title='Fuel Consumption Hwy (L/100km)'),
    size=alt.Size('CO2_Emissions', title='CO2 Emissions', scale=alt.Scale(range=[0, 1000])),
    color='Cylinders:N',
    tooltip=['Fuel_Consumption_City', 'Fuel_Consumption_Hwy', 'CO2_Emissions']
).properties(
    width=600,
    height=400
).interactive()

# Show the scatter plot
CO2_plot

alt.Chart(...)

In [44]:
numeric_columns = ['Engine_Size', 'Cylinders', 'Fuel_Consumption_City', 'Fuel_Consumption_Hwy', 
                   'Fuel_Consumption_Comb', 'Fuel_Consumption_Comb_MPG', 'CO2_Emissions', 
                   'Fuel_Consumption_Per_Cylinder', 'CO2_Emissions_Per_Cylinder', 'Engine_Size_Per_Cylinder']

repeated_circle = alt.Chart(car).mark_circle().encode(
    x=alt.X(alt.repeat('column'), type='quantitative'),
    y=alt.Y(alt.repeat('row'), type='quantitative')
).properties(
    width=200,
    height=200
).repeat(column=numeric_columns, row=numeric_columns)

repeated_circle

alt.RepeatChart(...)

In [45]:
num_column = ['CO2_Emissions', 'CO2_Emissions_Per_Cylinder']
cat_column = ['Make', 'Transmission']
light_blue = (64, 164, 223, 0.5)

repeated_box_plot = alt.Chart(car).mark_boxplot(opacity=0.5).encode(
    x=alt.X(alt.repeat('column'), type='quantitative'),
    y=alt.Y(alt.repeat('row'), type='nominal', title='')
).properties(
    width=400,
    height=400
).repeat(column=num_column, row=cat_column)

repeated_box_plot

alt.RepeatChart(...)

#### Splitting the Train and Test data for Model prediction:

In [46]:
X = car.drop(columns=['CO2_Emissions'])
y = car['CO2_Emissions']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_train

,Make,Model,Vehicle_Class,Engine_Size,Cylinders,Transmission,Fuel_Type,Fuel_Consumption_City,Fuel_Consumption_Hwy,Fuel_Consumption_Comb,Fuel_Consumption_Comb_MPG,Fuel_Consumption_Per_Cylinder,CO2_Emissions_Per_Cylinder,Engine_Size_Per_Cylinder
84,BMW,328i xDRIVE GRAN TURISMO,FULL-SIZE,2.0,4,A8,Z,10.6,7.2,9.1,31,2.275000,52.250000,0.500000
42,AUDI,Q7,SUV - STANDARD,3.0,6,AS8,Z,15.1,10.9,13.2,21,2.200000,50.666667,0.500000
381,FORD,F150 FFV,PICKUP TRUCK - STANDARD,5.0,8,AS6,X,16.4,12.3,14.6,19,1.825000,42.000000,0.625000
717,MAZDA,MAZDA6 (i-ELOOP),MID-SIZE,2.5,4,AS6,X,8.4,5.9,7.3,39,1.825000,42.000000,0.625000
684,LINCOLN,MKZ AWD,MID-SIZE,3.7,6,AS6,X,13.1,9.3,11.4,25,1.900000,43.666667,0.616667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,LAND ROVER,RANGE ROVER EVOQUE,SUV - SMALL,2.0,4,AS9,Z,11.3,7.9,9.8,29,2.450000,56.250000,0.500000
113,BMW,ACTIVEHYBRID 3,COMPACT,3.0,6,A8,Z,9.2,7.1,8.3,34,1.383333,31.833333,0.500000
96,BMW,435i xDRIVE COUPE,COMPACT,3.0,6,A8,Z,11.7,7.8,9.9,29,1.650000,38.000000,0.500000
106,BMW,650i xDRIVE CABRIOLET,SUBCOMPACT,4.4,8,A8,Z,15.0,9.8,12.7,22,1.587500,36.500000,0.550000


#### Preprocessing the features:

In [47]:
numeric_features = ['Engine_Size', 'Cylinders', 'Fuel_Consumption_City', 'Fuel_Consumption_Hwy', 
                    'Fuel_Consumption_Comb', 'Fuel_Consumption_Comb_MPG', 'Fuel_Consumption_Per_Cylinder', 
                    'CO2_Emissions_Per_Cylinder', 'Engine_Size_Per_Cylinder']

categorical_features = ['Make', 'Model', 'Vehicle_Class', 'Transmission', 'Fuel_Type']

In [48]:
numeric_transformer = make_pipeline(StandardScaler())

categorical_transformer = make_pipeline(OneHotEncoder(dtype=int, handle_unknown='ignore'))

preprocessor = make_column_transformer((numeric_transformer, numeric_features),
                                          (categorical_transformer, categorical_features),
                                          remainder='passthrough')

preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler())]),
                                 ['Engine_Size', 'Cylinders',
                                  'Fuel_Consumption_City',
                                  'Fuel_Consumption_Hwy',
                                  'Fuel_Consumption_Comb',
                                  'Fuel_Consumption_Comb_MPG',
                                  'Fuel_Consumption_Per_Cylinder',
                                  'CO2_Emissions_Per_Cylinder',
                                  'Engine_Size_Per_Cylinder']),
                                ('pipeline-2',
                                 Pipeline(steps=[('onehotencoder',
                                                  OneHotEncoder(dtype=<class 'int'>,
                                                                handle_unknown='ignore'))]),
                                 ['Make', 'Model', 'Vehicle_Class',
                                  'Transmission', 'Fuel_Type'])])

#### Cross-Validation using regression scoring technique and Model Selection:

In [49]:
dummy_reg = make_pipeline(preprocessor, DummyRegressor(strategy='median'))

dummy_score = cross_validate(dummy_reg, X_train, y_train, cv=5, return_train_score=True)
dummy_scores=pd.DataFrame(dummy_score)

dummy_scores

,fit_time,score_time,test_score,train_score
0,0.022287,0.006365,-0.002324,-0.007823
1,0.009462,0.004502,-0.029628,-0.013547
2,0.006479,0.003201,-0.007228,-0.006353
3,0.005306,0.002554,-0.000003,-0.009947
4,0.004572,0.002309,-0.031651,-0.013004


In [50]:
from sklearn.metrics import make_scorer

def mape(true, pred):
    return 100.0 * np.mean(np.abs((pred - true) / true))

mape_scorer = make_scorer(mape, greater_is_better=False)

In [51]:
scoring_dict = {"neg_mean_squared_error": "neg_mean_squared_error",
                "neg_root_mean_square_error": "neg_root_mean_squared_error",
                "neg_mean_absolute_error": "neg_mean_absolute_error",
                "r2": "r2",
                "mape_scorer": mape_scorer
               }

In [94]:
# LinerRegression (Ridge):

ridge_reg = make_pipeline(preprocessor, Ridge())

cv_results_ridge = cross_validate(ridge_reg, X_train, y_train, cv=5, return_train_score=True, scoring=scoring_dict)
ridge_score = pd.DataFrame(cv_results_ridge)
ridge_score

,fit_time,score_time,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_root_mean_square_error,train_neg_root_mean_square_error,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_r2,train_r2,test_mape_scorer,train_mape_scorer
0,0.024396,0.005494,-37.383834,-9.905824,-6.114232,-3.147352,-4.023706,-2.004735,0.989920,0.997625,-1.611918,-0.775046
1,0.010788,0.003538,-31.124081,-10.597740,-5.578896,-3.255417,-3.779734,-2.021176,0.992585,0.997382,-1.399699,-0.781249
2,0.007724,0.002839,-34.433530,-10.321065,-5.868009,-3.212641,-3.801051,-2.079575,0.990935,0.997513,-1.435816,-0.788782
3,0.006482,0.002439,-28.260693,-10.274520,-5.316079,-3.205389,-3.640174,-2.089796,0.992618,0.997515,-1.428013,-0.807371
4,0.006392,0.002415,-28.379995,-10.965084,-5.327288,-3.311357,-3.735858,-2.136261,0.994102,0.997183,-1.466402,-0.823135


In [53]:
ridge_mean = ridge_score.mean()
ridge_mean_df = pd.DataFrame(list(ridge_mean.items()), columns=['variable', 'mean_score'])
ridge_mean_df

,variable,mean_score
0,fit_time,0.012332
1,score_time,0.003950
2,test_neg_mean_squared_error,-31.916426
3,train_neg_mean_squared_error,-10.412847
4,test_neg_root_mean_square_error,-5.640901
5,train_neg_root_mean_square_error,-3.226431
6,test_neg_mean_absolute_error,-3.796105
7,train_neg_mean_absolute_error,-2.066309
8,test_r2,0.992032
9,train_r2,0.997444


In [95]:
# RandomForest Regression:

rfr = make_pipeline(preprocessor, RandomForestRegressor())

rfr_results = cross_validate(rfr, X_train, y_train, cv=5, return_train_score=True, scoring=scoring_dict)
rfr_score = pd.DataFrame(rfr_results)
rfr_score

,fit_time,score_time,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_root_mean_square_error,train_neg_root_mean_square_error,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_r2,train_r2,test_mape_scorer,train_mape_scorer
0,0.372667,0.006652,-68.597181,-5.194680,-8.282342,-2.279184,-2.159883,-0.531598,0.981504,0.998755,-0.897078,-0.206906
1,0.331746,0.006061,-66.438280,-3.663413,-8.150968,-1.914005,-1.893977,-0.560513,0.984173,0.999095,-0.697256,-0.224637
2,0.333682,0.006306,-77.138051,-5.580351,-8.782827,-2.362277,-2.094386,-0.664091,0.979694,0.998655,-0.898189,-0.254121
3,0.336116,0.006276,-15.860425,-5.568039,-3.982515,-2.359669,-1.283059,-0.652299,0.995857,0.998653,-0.503736,-0.257605
4,0.330612,0.006131,-47.312206,-7.065378,-6.878387,-2.658078,-1.881176,-0.643485,0.990168,0.998185,-0.595891,-0.258695


In [60]:
rfr_mean = rfr_score.mean()
rfr_mean_df = pd.DataFrame(list(rfr_mean.items()), columns=['variable', 'mean_score'])
rfr_mean_df

,variable,mean_score
0,fit_time,0.336942
1,score_time,0.006260
2,test_neg_mean_squared_error,-64.080543
3,train_neg_mean_squared_error,-6.049572
4,test_neg_root_mean_square_error,-7.828506
5,train_neg_root_mean_square_error,-2.451866
6,test_neg_mean_absolute_error,-2.003281
7,train_neg_mean_absolute_error,-0.640989
8,test_r2,0.984097
9,train_r2,0.998515


In [96]:
# Standard Vector Regression:

svr = make_pipeline(preprocessor, SVR())

svr_results = cross_validate(svr, X_train, y_train, cv=5, return_train_score=True, scoring=scoring_dict)
svr_score = pd.DataFrame(svr_results)
svr_score

,fit_time,score_time,test_neg_mean_squared_error,train_neg_mean_squared_error,test_neg_root_mean_square_error,train_neg_root_mean_square_error,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_r2,train_r2,test_mape_scorer,train_mape_scorer
0,0.086116,0.009895,-1110.787564,-1093.857227,-33.328480,-33.073512,-17.808290,-18.625080,0.700490,0.737767,-6.858828,-7.086325
1,0.022818,0.005719,-1217.989179,-1121.269795,-34.899702,-33.485367,-19.811651,-18.962972,0.709843,0.722968,-7.555372,-7.240134
2,0.021535,0.005686,-916.195217,-1167.305944,-30.268717,-34.165859,-17.004331,-19.311632,0.758813,0.718741,-6.019264,-7.434834
3,0.021717,0.005945,-909.880609,-1169.610390,-30.164227,-34.199567,-17.019172,-19.374475,0.762315,0.717112,-6.842387,-7.340662
4,0.021730,0.005664,-1421.867161,-1111.216138,-37.707654,-33.334909,-22.595921,-18.697391,0.704519,0.714496,-8.572783,-7.125474


In [61]:
svr_mean = svr_score.mean()
svr_mean_df = pd.DataFrame(list(svr_mean.items()), columns=['variable', 'mean_score'])
svr_mean_df

,variable,mean_score
0,fit_time,0.027123
1,score_time,0.006302
2,test_neg_mean_squared_error,-1115.343946
3,train_neg_mean_squared_error,-1132.651899
4,test_neg_root_mean_square_error,-33.273756
5,train_neg_root_mean_square_error,-33.651843
6,test_neg_mean_absolute_error,-18.847873
7,train_neg_mean_absolute_error,-18.994310
8,test_r2,0.727196
9,train_r2,0.722217


In [63]:
ridge_mean_df['Model'] = 'Ridge Regression'
rfr_mean_df['Model'] = 'Random Forest Regression'
svr_mean_df['Model'] = 'Support Vector Regression'


mlmodel_df = pd.concat([ridge_mean_df, rfr_mean_df, svr_mean_df], ignore_index=True)
mlmodel_df

,variable,mean_score,Model
0,fit_time,0.012332,Ridge Regression
1,score_time,0.003950,Ridge Regression
2,test_neg_mean_squared_error,-31.916426,Ridge Regression
3,train_neg_mean_squared_error,-10.412847,Ridge Regression
4,test_neg_root_mean_square_error,-5.640901,Ridge Regression
5,train_neg_root_mean_square_error,-3.226431,Ridge Regression
6,test_neg_mean_absolute_error,-3.796105,Ridge Regression
7,train_neg_mean_absolute_error,-2.066309,Ridge Regression
8,test_r2,0.992032,Ridge Regression
9,train_r2,0.997444,Ridge Regression


In [69]:
mlmodel_tidy = mlmodel_df.pivot_table(index='variable', columns='Model', values='mean_score').reset_index()

# Melt the dataframe to long format
mlmodel_tidy = mlmodel_tidy.melt(id_vars='variable', var_name='Model', value_name='mean_score')

# Filter the dataframe for test_mape_scorer and train_mape_scorer metrics
mape_df = mlmodel_tidy[mlmodel_tidy['variable'].isin(['test_mape_scorer', 'train_mape_scorer'])]

chart = alt.Chart(mape_df).mark_bar().encode(
    y='Model:N',
    x='mean_score:Q',
    color='Model:N',
    column='variable:N',
    tooltip=['Model:N', 'mean_score:Q']
).properties(
    width=200,
    height=200
)

chart

alt.Chart(...)

#### Hyperparameter Optimization for SVR Model:

In [80]:
svr_pipeline = make_pipeline(
    preprocessor,
    SVR()
)

param_dist = {
    'svr__C': reciprocal(0.001, 1000), 
    'svr__gamma': reciprocal(0.0001, 1),
    'svr__epsilon': uniform(0, 1)
}

# Create RandomizedSearchCV
random_search_svr = RandomizedSearchCV(
    svr_pipeline,
    param_distributions=param_dist,
    n_iter=50, 
    cv=5,  
    scoring=mape_scorer,
    verbose=1,
    n_jobs=-1,
    random_state=123,
)

random_search_svr.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['Engine_Size',
                                                                                'Cylinders',
                                                                                'Fuel_Consumption_City',
                                                                                'Fuel_Consumption_Hwy',
                                                                                'Fuel_Consumption_Comb',
                                                                                'Fuel_Consumption_Comb_MPG',
                                                                                'Fuel_Consumption_Per_Cylinde...
                   param_distributions={'svr__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x1637716f0>,
                                        'svr__epsilon': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x16407ef80>,
                                        'svr__gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x164033cd0>},
                   random_state=123,
                   scoring=make_scorer(mape, greater_is_better=False),
                   verbose=1)

In [81]:
#let's get us a best hyperparameters

optimal_parameters = random_search_svr.best_params_
optimal_score = random_search_svr.best_score_

optimal_parameters, optimal_score

({'svr__C': 766.6289057556017,
  'svr__epsilon': 0.6848297385848633,
  'svr__gamma': 0.008389336334792568},
 -0.40473133530019056)

In [82]:
training_score = random_search_svr.score(X_train, y_train)
testing_score = random_search_svr.score(X_test, y_test)

training_score, testing_score

(-0.20519994461154104, -0.3774976113434244)

#### How well did our Model Performed? Let's compute the scoring metrics for regression:

In [86]:
y_pred = random_search_svr.predict(X_test)

df_pred_actual = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

scatter_plot = alt.Chart(df_pred_actual).mark_circle().encode(
    x='Actual',
    y='Predicted',
    color=alt.ColorValue('steelblue'),
    tooltip=['Actual', 'Predicted']
).properties(
    width=500,
    height=400
)

diagonal_line = alt.Chart(pd.DataFrame({'x': [min(y_test), max(y_test)], 'y': [min(y_test), max(y_test)]})).mark_line(
    color='red',
    strokeDash=[3,3]
).encode(
    x='x',
    y='y'
)

scatter_with_diagonal = scatter_plot + diagonal_line

scatter_with_diagonal.interactive()

alt.LayerChart(...)

In [91]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [92]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
print(f'R^2 Score: {r2:.2f}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

Mean Absolute Error (MAE): 0.95
Mean Squared Error (MSE): 1.74
Root Mean Squared Error (RMSE): 1.32
R^2 Score: 1.00
Mean Absolute Percentage Error (MAPE): 0.38%
